In [ ]:
import numpy as np
import math
import time
import torch
import os
from torch.autograd import Variable
import matplotlib.pyplot as plt
from Modules.training_functions import *
from Modules.pennylane_functions import *

# if gpu available, set device to gpu
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using the GPU")
else:
    device = torch.device("cpu")
    print("WARNING: Could not find GPU, using the CPU")
T=5
all=np.load(f'Data/dataset_ld_{ld_dim}_{9}.npy')
all=all[:200]
for i in range(9):
    x=np.load(f'Data/dataset_ld_{ld_dim}_{i}.npy')
    all=np.concatenate((x[:200],all))

mnist_images=all
np.random.shuffle(mnist_images)
mnist_images = torch.tensor(mnist_images).to(device)
#qc_array=np.array([0,48,60,63]) 4
# make dataloader
data_loader = torch.utils.data.DataLoader(mnist_images, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
qc_array=np.array([0,256,448,496,510])
min_array=np.array([0.05,0.01,0.005])
layer_array=np.array([5,10,20,50]) 
print(NUM_QUBITS)
print(T)
zero = torch.zeros(BATCH_SIZE, 2**NUM_QUBITS-ld_dim).to(device)
for layer_indx in range(len(layer_array)):
    n_layer=layer_array[layer_indx]
    for q_indx in range(len(qc_array)):
        qc=qc_array[q_indx]
        for min_indx in range(len(min_array)):
            min_b=min_array[min_indx]

            betas      = np.insert(np.linspace(10e-8,min_b, T), 0, 0)
            print(np.shape(betas))
            alphas     = 1 - betas
            alphas_bar = np.cumprod(alphas)
            pi         = math.pi
            betas      = torch.tensor(betas).float().to(device)
            alphas     = torch.tensor(alphas).float().to(device)
            alphas_bar = torch.tensor(alphas_bar).float().to(device)
            theta_1    = Variable(torch.rand((n_layer*3*NUM_QUBITS+n_layer*3*(NUM_QUBITS)), device = device), requires_grad=True)
            optimizer = torch.optim.Adam([theta_1], lr = LEARNING_RATE)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = SCHEDULER_PATIENCE, gamma = SCHEDULER_GAMMA, verbose = False)
            trained_thetas_1 = []
            loss_history = []
            best_loss = 1e10

            for epoch in range(NUM_EPOCHS):
                print(epoch)

                t0 = time.time()
                num_batch=0
                tot_loss=0

                for image_batch in data_loader:

                    # extract batch of random times and betas
                    t = torch.randint(0, T, size = (BATCH_SIZE, ), device=device)
                    betas_batch = betas[t].to(device)
                    alphas_batch=alphas_bar[t].to(device)

                    # assemble input at t add noise (t+1)
                    target_batch = assemble_input(image_batch, t, alphas_bar,ld_dim ,device)
                    input_batch  = noise_step(target_batch, t+1, betas,ld_dim, device)
                    target_batch = target_batch / torch.norm(target_batch, dim = 1).view(-1, 1)
                    input_batch  = input_batch / torch.norm(input_batch, dim = 1).view(-1, 1)
                    

                    # concatenate the two tensors along the second dimension
                    input_batch = torch.cat((input_batch, zero), dim=1)
                    target_batch = torch.cat((target_batch, zero), dim=1)
                    # Feed to circuit, compute the loss and update the weights
                    num_batch+=1
                    loss = loss_fn_aq(qc,theta_1,n_layer, input_batch, target_batch)
                    tot_loss+=loss.item()
                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()

                # append parameters and print loss
                trained_thetas_1.append(theta_1.cpu().clone().detach().numpy())

                loss_history.append(tot_loss/num_batch)
                if loss.item()< best_loss:
                    best_loss=loss.item()

                # implement learning rate scheduler
                scheduler.step()


            # print every epoch
                print(f'T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={((time.time()-t0)*(NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))/60:.2f} min{min_b} nl{n_layer} QC{qc}')
                #print(f'T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={(((NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))):.2f} min{min_b} nl{n_layer}')
                
            np.save(f'ancilla_{Q_ANCILLA}/all_thetas_T{T}_nl{n_layer}_min{min_b}_qc{qc}_{Q_ANCILLA}_ld{ld_dim}.npy',trained_thetas_1)
            np.save(f'ancilla_{Q_ANCILLA}/all_loss__T{T}_nl{n_layer}_min{min_b}_qc{qc}_ancilla{Q_ANCILLA}_ld{ld_dim}.npy',loss_history)

9
5
(6,)
0
T=5 Epoch: 1/40 - Loss: 0.8266 b_loss=0.8266 - T: 28.81s/epoch ,tempo_previto=1152.08 min0.05 nl5 QC0
1
T=5 Epoch: 2/40 - Loss: 0.6085 b_loss=0.6085 - T: 32.01s/epoch ,tempo_previto=1279.44 min0.05 nl5 QC0
2
T=5 Epoch: 3/40 - Loss: 0.4835 b_loss=0.4835 - T: 32.00s/epoch ,tempo_previto=1278.40 min0.05 nl5 QC0
3
T=5 Epoch: 4/40 - Loss: 0.4091 b_loss=0.4091 - T: 31.27s/epoch ,tempo_previto=1248.57 min0.05 nl5 QC0
4
T=5 Epoch: 5/40 - Loss: 0.3756 b_loss=0.3756 - T: 30.41s/epoch ,tempo_previto=1213.77 min0.05 nl5 QC0
5
T=5 Epoch: 6/40 - Loss: 0.3548 b_loss=0.3548 - T: 30.63s/epoch ,tempo_previto=1222.00 min0.05 nl5 QC0
6
T=5 Epoch: 7/40 - Loss: 0.3515 b_loss=0.3515 - T: 30.51s/epoch ,tempo_previto=1217.03 min0.05 nl5 QC0
7
T=5 Epoch: 8/40 - Loss: 0.3338 b_loss=0.3338 - T: 30.50s/epoch ,tempo_previto=1216.11 min0.05 nl5 QC0
8
T=5 Epoch: 9/40 - Loss: 0.3315 b_loss=0.3315 - T: 30.59s/epoch ,tempo_previto=1218.86 min0.05 nl5 QC0
9
T=5 Epoch: 10/40 - Loss: 0.3165 b_loss=0.3165 - T: 30

T=5 Epoch: 39/40 - Loss: 0.2130 b_loss=0.2046 - T: 30.52s/epoch ,tempo_previto=1099.17 min0.01 nl5 QC0
39
T=5 Epoch: 40/40 - Loss: 0.2145 b_loss=0.2046 - T: 30.63s/epoch ,tempo_previto=1102.53 min0.01 nl5 QC0
(6,)
0
T=5 Epoch: 1/40 - Loss: 0.8838 b_loss=0.8838 - T: 30.81s/epoch ,tempo_previto=1026.58 min0.005 nl5 QC0
1
T=5 Epoch: 2/40 - Loss: 0.6554 b_loss=0.6554 - T: 30.50s/epoch ,tempo_previto=1015.61 min0.005 nl5 QC0
2
T=5 Epoch: 3/40 - Loss: 0.4824 b_loss=0.4824 - T: 30.60s/epoch ,tempo_previto=1018.46 min0.005 nl5 QC0
3
T=5 Epoch: 4/40 - Loss: 0.3890 b_loss=0.3890 - T: 30.52s/epoch ,tempo_previto=1015.42 min0.005 nl5 QC0
4
T=5 Epoch: 5/40 - Loss: 0.3156 b_loss=0.3156 - T: 30.48s/epoch ,tempo_previto=1013.56 min0.005 nl5 QC0
5
T=5 Epoch: 6/40 - Loss: 0.3149 b_loss=0.3149 - T: 30.62s/epoch ,tempo_previto=1017.60 min0.005 nl5 QC0
6
T=5 Epoch: 7/40 - Loss: 0.2974 b_loss=0.2974 - T: 30.43s/epoch ,tempo_previto=1010.94 min0.005 nl5 QC0
7
T=5 Epoch: 8/40 - Loss: 0.2850 b_loss=0.2850 - T:

T=5 Epoch: 36/40 - Loss: 0.2957 b_loss=0.2860 - T: 30.66s/epoch ,tempo_previto=1187.70 min0.05 nl5 QC256
36
T=5 Epoch: 37/40 - Loss: 0.3062 b_loss=0.2860 - T: 31.89s/epoch ,tempo_previto=1234.82 min0.05 nl5 QC256
37
T=5 Epoch: 38/40 - Loss: 0.3162 b_loss=0.2860 - T: 30.89s/epoch ,tempo_previto=1195.57 min0.05 nl5 QC256
38
T=5 Epoch: 39/40 - Loss: 0.2919 b_loss=0.2860 - T: 30.97s/epoch ,tempo_previto=1197.93 min0.05 nl5 QC256
39
T=5 Epoch: 40/40 - Loss: 0.3004 b_loss=0.2860 - T: 30.86s/epoch ,tempo_previto=1193.18 min0.05 nl5 QC256
(6,)
0
T=5 Epoch: 1/40 - Loss: 0.8185 b_loss=0.8185 - T: 30.89s/epoch ,tempo_previto=1111.64 min0.01 nl5 QC256
1
T=5 Epoch: 2/40 - Loss: 0.6093 b_loss=0.6093 - T: 30.56s/epoch ,tempo_previto=1099.22 min0.01 nl5 QC256
2
T=5 Epoch: 3/40 - Loss: 0.4631 b_loss=0.4631 - T: 30.84s/epoch ,tempo_previto=1108.53 min0.01 nl5 QC256
3
T=5 Epoch: 4/40 - Loss: 0.3658 b_loss=0.3658 - T: 30.84s/epoch ,tempo_previto=1108.26 min0.01 nl5 QC256
4
T=5 Epoch: 5/40 - Loss: 0.3099 b

T=5 Epoch: 33/40 - Loss: 0.2134 b_loss=0.2053 - T: 30.97s/epoch ,tempo_previto=994.72 min0.005 nl5 QC256
33
T=5 Epoch: 34/40 - Loss: 0.2118 b_loss=0.2053 - T: 30.92s/epoch ,tempo_previto=992.49 min0.005 nl5 QC256
34
T=5 Epoch: 35/40 - Loss: 0.2159 b_loss=0.2053 - T: 30.91s/epoch ,tempo_previto=991.82 min0.005 nl5 QC256
35
T=5 Epoch: 36/40 - Loss: 0.2086 b_loss=0.2053 - T: 31.17s/epoch ,tempo_previto=999.48 min0.005 nl5 QC256
36
T=5 Epoch: 37/40 - Loss: 0.2130 b_loss=0.2053 - T: 31.09s/epoch ,tempo_previto=996.37 min0.005 nl5 QC256
37
T=5 Epoch: 38/40 - Loss: 0.2098 b_loss=0.2053 - T: 30.97s/epoch ,tempo_previto=991.98 min0.005 nl5 QC256
38
T=5 Epoch: 39/40 - Loss: 0.2157 b_loss=0.2053 - T: 31.09s/epoch ,tempo_previto=995.47 min0.005 nl5 QC256
39
T=5 Epoch: 40/40 - Loss: 0.2157 b_loss=0.2053 - T: 31.20s/epoch ,tempo_previto=998.45 min0.005 nl5 QC256
(6,)
0
T=5 Epoch: 1/40 - Loss: 0.8850 b_loss=0.8850 - T: 30.93s/epoch ,tempo_previto=1195.27 min0.05 nl5 QC448
1
T=5 Epoch: 2/40 - Loss: 0.

T=5 Epoch: 30/40 - Loss: 0.2288 b_loss=0.2288 - T: 30.97s/epoch ,tempo_previto=1078.94 min0.01 nl5 QC448
30
T=5 Epoch: 31/40 - Loss: 0.2362 b_loss=0.2288 - T: 30.66s/epoch ,tempo_previto=1067.58 min0.01 nl5 QC448
31
T=5 Epoch: 32/40 - Loss: 0.2288 b_loss=0.2288 - T: 30.83s/epoch ,tempo_previto=1072.83 min0.01 nl5 QC448
32
T=5 Epoch: 33/40 - Loss: 0.2316 b_loss=0.2288 - T: 30.77s/epoch ,tempo_previto=1070.31 min0.01 nl5 QC448
33
T=5 Epoch: 34/40 - Loss: 0.2272 b_loss=0.2272 - T: 31.01s/epoch ,tempo_previto=1078.21 min0.01 nl5 QC448
34
T=5 Epoch: 35/40 - Loss: 0.2284 b_loss=0.2272 - T: 30.46s/epoch ,tempo_previto=1058.46 min0.01 nl5 QC448
35
T=5 Epoch: 36/40 - Loss: 0.2371 b_loss=0.2272 - T: 30.59s/epoch ,tempo_previto=1062.39 min0.01 nl5 QC448
36
T=5 Epoch: 37/40 - Loss: 0.2318 b_loss=0.2272 - T: 30.82s/epoch ,tempo_previto=1070.00 min0.01 nl5 QC448
37
T=5 Epoch: 38/40 - Loss: 0.2325 b_loss=0.2272 - T: 31.47s/epoch ,tempo_previto=1091.93 min0.01 nl5 QC448
38
T=5 Epoch: 39/40 - Loss: 0.2

...

T=5 Epoch: 37/40 - Loss: 0.0969 b_loss=0.0969 - T: 121.85s/epoch ,tempo_previto=1305.85 min0.005 nl20 QC496
37
T=5 Epoch: 38/40 - Loss: 0.1001 b_loss=0.0969 - T: 122.10s/epoch ,tempo_previto=1306.46 min0.005 nl20 QC496
38
T=5 Epoch: 39/40 - Loss: 0.1045 b_loss=0.0969 - T: 122.09s/epoch ,tempo_previto=1304.30 min0.005 nl20 QC496
39
T=5 Epoch: 40/40 - Loss: 0.0969 b_loss=0.0969 - T: 122.11s/epoch ,tempo_previto=1302.46 min0.005 nl20 QC496
(6,)
0
T=5 Epoch: 1/40 - Loss: 0.5902 b_loss=0.5902 - T: 121.40s/epoch ,tempo_previto=2102.32 min0.05 nl20 QC510
1
T=5 Epoch: 2/40 - Loss: 0.3038 b_loss=0.3038 - T: 122.44s/epoch ,tempo_previto=2118.27 min0.05 nl20 QC510
2
T=5 Epoch: 3/40 - Loss: 0.2693 b_loss=0.2693 - T: 122.43s/epoch ,tempo_previto=2115.96 min0.05 nl20 QC510
3
T=5 Epoch: 4/40 - Loss: 0.2572 b_loss=0.2572 - T: 122.14s/epoch ,tempo_previto=2108.94 min0.05 nl20 QC510
4
T=5 Epoch: 5/40 - Loss: 0.2455 b_loss=0.2455 - T: 121.68s/epoch ,tempo_previto=2099.04 min0.05 nl20 QC510
5
T=5 Epoch: 6

T=5 Epoch: 32/40 - Loss: 0.1662 b_loss=0.1639 - T: 122.79s/epoch ,tempo_previto=1653.63 min0.01 nl20 QC510
32
T=5 Epoch: 33/40 - Loss: 0.1687 b_loss=0.1639 - T: 122.48s/epoch ,tempo_previto=1647.39 min0.01 nl20 QC510
33
T=5 Epoch: 34/40 - Loss: 0.1658 b_loss=0.1639 - T: 121.68s/epoch ,tempo_previto=1634.59 min0.01 nl20 QC510
34
T=5 Epoch: 35/40 - Loss: 0.1700 b_loss=0.1639 - T: 121.82s/epoch ,tempo_previto=1634.39 min0.01 nl20 QC510
35
T=5 Epoch: 36/40 - Loss: 0.1685 b_loss=0.1639 - T: 122.30s/epoch ,tempo_previto=1638.82 min0.01 nl20 QC510
36
T=5 Epoch: 37/40 - Loss: 0.1661 b_loss=0.1639 - T: 122.14s/epoch ,tempo_previto=1634.58 min0.01 nl20 QC510
37
T=5 Epoch: 38/40 - Loss: 0.1740 b_loss=0.1639 - T: 121.69s/epoch ,tempo_previto=1626.58 min0.01 nl20 QC510
38
T=5 Epoch: 39/40 - Loss: 0.1648 b_loss=0.1639 - T: 122.13s/epoch ,tempo_previto=1630.39 min0.01 nl20 QC510
39
T=5 Epoch: 40/40 - Loss: 0.1640 b_loss=0.1639 - T: 121.99s/epoch ,tempo_previto=1626.56 min0.01 nl20 QC510
(6,)
0
T=5 Ep

T=5 Epoch: 27/40 - Loss: 0.0594 b_loss=0.0594 - T: 300.74s/epoch ,tempo_previto=2872.04 min0.05 nl50 QC0
27
T=5 Epoch: 28/40 - Loss: 0.0627 b_loss=0.0594 - T: 303.35s/epoch ,tempo_previto=2891.97 min0.05 nl50 QC0
28
T=5 Epoch: 29/40 - Loss: 0.0640 b_loss=0.0594 - T: 301.99s/epoch ,tempo_previto=2873.94 min0.05 nl50 QC0
29
T=5 Epoch: 30/40 - Loss: 0.0653 b_loss=0.0594 - T: 304.30s/epoch ,tempo_previto=2890.82 min0.05 nl50 QC0
30
T=5 Epoch: 31/40 - Loss: 0.0564 b_loss=0.0564 - T: 306.44s/epoch ,tempo_previto=2906.06 min0.05 nl50 QC0
31
T=5 Epoch: 32/40 - Loss: 0.0639 b_loss=0.0564 - T: 305.13s/epoch ,tempo_previto=2888.59 min0.05 nl50 QC0
32
T=5 Epoch: 33/40 - Loss: 0.0678 b_loss=0.0564 - T: 306.97s/epoch ,tempo_previto=2900.90 min0.05 nl50 QC0
33
T=5 Epoch: 34/40 - Loss: 0.0654 b_loss=0.0564 - T: 307.62s/epoch ,tempo_previto=2901.90 min0.05 nl50 QC0
34
T=5 Epoch: 35/40 - Loss: 0.0572 b_loss=0.0564 - T: 310.16s/epoch ,tempo_previto=2920.68 min0.05 nl50 QC0
35
T=5 Epoch: 36/40 - Loss: 0.0

T=5 Epoch: 24/40 - Loss: 0.0237 b_loss=0.0231 - T: 304.81s/epoch ,tempo_previto=894.10 min0.005 nl50 QC0
24
T=5 Epoch: 25/40 - Loss: 0.0241 b_loss=0.0231 - T: 308.25s/epoch ,tempo_previto=899.07 min0.005 nl50 QC0
25
T=5 Epoch: 26/40 - Loss: 0.0235 b_loss=0.0231 - T: 303.97s/epoch ,tempo_previto=881.52 min0.005 nl50 QC0
26
T=5 Epoch: 27/40 - Loss: 0.0238 b_loss=0.0231 - T: 300.89s/epoch ,tempo_previto=867.57 min0.005 nl50 QC0
27
T=5 Epoch: 28/40 - Loss: 0.0230 b_loss=0.0230 - T: 301.75s/epoch ,tempo_previto=865.02 min0.005 nl50 QC0
28
T=5 Epoch: 29/40 - Loss: 0.0235 b_loss=0.0230 - T: 302.85s/epoch ,tempo_previto=863.12 min0.005 nl50 QC0
29
T=5 Epoch: 30/40 - Loss: 0.0225 b_loss=0.0225 - T: 301.99s/epoch ,tempo_previto=855.65 min0.005 nl50 QC0
30
T=5 Epoch: 31/40 - Loss: 0.0234 b_loss=0.0225 - T: 303.29s/epoch ,tempo_previto=854.28 min0.005 nl50 QC0
31
T=5 Epoch: 32/40 - Loss: 0.0225 b_loss=0.0225 - T: 302.61s/epoch ,tempo_previto=847.30 min0.005 nl50 QC0
32
T=5 Epoch: 33/40 - Loss: 0.0

T=5 Epoch: 20/40 - Loss: 0.0318 b_loss=0.0303 - T: 307.49s/epoch ,tempo_previto=1742.45 min0.01 nl50 QC256
20
T=5 Epoch: 21/40 - Loss: 0.0303 b_loss=0.0303 - T: 305.72s/epoch ,tempo_previto=1727.32 min0.01 nl50 QC256
21
T=5 Epoch: 22/40 - Loss: 0.0307 b_loss=0.0303 - T: 305.66s/epoch ,tempo_previto=1721.88 min0.01 nl50 QC256
22
T=5 Epoch: 23/40 - Loss: 0.0292 b_loss=0.0292 - T: 305.05s/epoch ,tempo_previto=1713.39 min0.01 nl50 QC256
23
T=5 Epoch: 24/40 - Loss: 0.0285 b_loss=0.0285 - T: 304.21s/epoch ,tempo_previto=1703.57 min0.01 nl50 QC256
24
T=5 Epoch: 25/40 - Loss: 0.0297 b_loss=0.0285 - T: 303.24s/epoch ,tempo_previto=1693.10 min0.01 nl50 QC256
25
T=5 Epoch: 26/40 - Loss: 0.0293 b_loss=0.0285 - T: 305.50s/epoch ,tempo_previto=1700.63 min0.01 nl50 QC256
26
T=5 Epoch: 27/40 - Loss: 0.0277 b_loss=0.0277 - T: 305.02s/epoch ,tempo_previto=1692.87 min0.01 nl50 QC256
27
T=5 Epoch: 28/40 - Loss: 0.0281 b_loss=0.0277 - T: 304.56s/epoch ,tempo_previto=1685.26 min0.01 nl50 QC256
28
T=5 Epoch: